In [2]:
!pip install -q -U tensorflow_datasets
!pip install tensorflow-metadata

In [3]:
! python -m tensorflow_datasets.scripts.download_and_prepare --register_checksums --datasets=oxford_flowers102

2020-06-13 10:41:02.579147: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
I0613 10:41:06.832287 140022945085312 download_and_prepare.py:194] Running download_and_prepare for dataset(s):
oxford_flowers102
I0613 10:41:06.833777 140022945085312 dataset_info.py:361] Load dataset info from /root/tensorflow_datasets/oxford_flowers102/2.1.0
I0613 10:41:06.837062 140022945085312 download_and_prepare.py:133] download_and_prepare for dataset oxford_flowers102/2.1.0...
I0613 10:41:06.837439 140022945085312 dataset_builder.py:282] Reusing dataset oxford_flowers102 (/root/tensorflow_datasets/oxford_flowers102/2.1.0)
name: "oxford_flowers102"
description: "The Oxford Flowers 102 dataset is a consistent of 102 flower categories commonly occurring\nin the United Kingdom. Each class consists of between 40 and 258 images. The images have\nlarge scale, pose and light variations. In addition, there are categories that have large\nvari

In [4]:
import tensorflow_datasets as tfds
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation, MaxPooling2D, Flatten, Dropout, AveragePooling2D
from tensorflow.keras.optimizers import Adam
import numpy as np


Using TensorFlow backend.


In [0]:
test, train, validate = tfds.load('oxford_flowers102', split=['train','test','validation'], as_supervised=True)

In [9]:
test

<PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [10]:
validate

<PrefetchDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [0]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [0]:

ntrain=sum(1 for _ in train)
ntest=sum(1 for _ in test)
nval=sum(1 for _ in  validate)

In [13]:
nval

1020

In [0]:
def normalize_img(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    image = tf.image.resize(image, (128,128))
    return image, label


In [0]:
mapped_train = train.map(normalize_img)
cached_train = mapped_train.cache()
batch_train = cached_train.repeat(20).batch(32)
tuned_train = batch_train.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
mapped_test = test.map(normalize_img)
repeated_test = mapped_test.repeat(20).batch(32)
cached_test = repeated_test.cache()
tuned_test = cached_test.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
mapped_validate = validate.map(normalize_img)
repeated_validate = mapped_validate.repeat(20).batch(32)
cached_validate = repeated_validate.cache()
tuned_validate = cached_validate.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
from keras.layers import AveragePooling2D

In [0]:
tf.keras.backend.clear_session()

In [0]:
numpy_train=tfds.as_numpy(tuned_train)
numpy_test=tfds.as_numpy(tuned_test)
numpy_validate=tfds.as_numpy(tuned_validate)
model=Sequential()
model.add(Conv2D(64, (3,3),input_shape=(128,128,3),strides=(1,1) , activation='relu'))
model.add(Conv2D(64, (3,3), padding='same',strides=(1,1), activation='relu'))
model.add(AveragePooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3,3), padding='same',strides=(1,1), activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3,3), padding='same',strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(256, (3,3), padding='same',strides=(1,1), activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(256, (3,3), padding='same',strides=(1,1), activation='relu'))
model.add(Flatten())
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dense(102, activation='softmax'))


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 64)      36928     
_________________________________________________________________
average_pooling2d_1 (Average (None, 63, 63, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 63, 63, 128)       73856     
_________________________________________________________________
dropout_2 (Dropout)          (None, 63, 63, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 63, 63, 128)      

In [0]:
model.compile(loss='sparse_categorical_crossentropy',  optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [23]:
model.fit(numpy_train,epochs=20,steps_per_epoch=ntrain//32, validation_data=numpy_validate, validation_steps=nval//32)

Epoch 1/20
192/192 [==============================] - 46s 238ms/step - loss: 4.4856 - accuracy: 0.0371 - val_loss: 4.7854 - val_accuracy: 0.0101
Epoch 2/20
192/192 [==============================] - 36s 187ms/step - loss: 4.3663 - accuracy: 0.0469 - val_loss: 4.7551 - val_accuracy: 0.0232
Epoch 3/20
192/192 [==============================] - 36s 189ms/step - loss: 3.9181 - accuracy: 0.0874 - val_loss: 4.1791 - val_accuracy: 0.0544
Epoch 4/20
192/192 [==============================] - 36s 189ms/step - loss: 3.5898 - accuracy: 0.1388 - val_loss: 3.6996 - val_accuracy: 0.0776
Epoch 5/20
192/192 [==============================] - 36s 189ms/step - loss: 3.1954 - accuracy: 0.2109 - val_loss: 3.6699 - val_accuracy: 0.1220
Epoch 6/20
192/192 [==============================] - 36s 189ms/step - loss: 2.7444 - accuracy: 0.3058 - val_loss: 4.1033 - val_accuracy: 0.1573
Epoch 7/20
192/192 [==============================] - 36s 188ms/step - loss: 2.1667 - accuracy: 0.4287 - val_loss: 4.2251 - val_ac

In [0]:
scores = model.evaluate_generator(numpy_test, steps=30)

In [25]:
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 11.368631362915039
Test accuracy: 0.15312500298023224
